# RBI Monthly Retail Payments Data Ingestion & Normalization

**Dataset**: Monthly Entity-wise Retail Payment Statistics  
**Publisher**: Reserve Bank of India (RBI)  

This notebook ingests raw monthly Excel files published by the RBI, covering multiple retail payment systems (UPI, IMPS, NETC, NFS, AePS, CTS, BBPS) at the bank/entity level.  

The goal is to:  
- Parse all monthly files across multiple sheets  
- Handle heterogeneous sheet layouts  
- Normalize role-based transaction volumes and values into a single long-format dataset  
- Enable downstream aggregation, market share computation, and anomaly detection  

This notebook focuses exclusively on **data engineering and preprocessing**. No analytics or modeling is performed here.

## Data Source

Entity-wise retail payment statistics are published by RBI at:

🔗 https://www.rbi.org.in/Scripts/EntityWiseRetailStatistics.aspx

Characteristics:
- One Excel file per month
- Multiple payment systems per file
- Different column layouts across instruments
- Volume reported in **Lakhs**
- Value reported in **Crores**

Data coverage used in this project:
- **April 2024 – October 2025**


In [29]:
import pandas as pd
import os
import re


## Extracting Month Information

The RBI files do not contain a clean month column.
Hence, month and year are extracted from the filename itself.


In [30]:
def extract_month_year(filename):
    """
    Extract Month and Year from RBI/NPCI filenames.

    Example:
        'UPI_April_2024.xlsx' → 'April 2024'
    """
    base = filename.lower()

    month_map = {
        'jan': 'January', 'january': 'January',
        'feb': 'February', 'february': 'February',
        'mar': 'March', 'march': 'March',
        'apr': 'April', 'april': 'April',
        'may': 'May',
        'jun': 'June', 'june': 'June',
        'jul': 'July', 'july': 'July',
        'aug': 'August', 'august': 'August',
        'sep': 'September', 'sept': 'September', 'september': 'September',
        'oct': 'October', 'october': 'October',
        'nov': 'November', 'november': 'November',
        'dec': 'December', 'december': 'December'
    }

    month_found = next((v for k, v in month_map.items() if k in base), None)
    year_match = re.search(r'(20\d{2}|\d{2})', base)

    year = (
        '20' + year_match.group(0)
        if year_match and len(year_match.group(0)) == 2
        else year_match.group(0) if year_match else None
    )

    return f"{month_found} {year}" if month_found and year else None


## Parsing Entity-wise Excel Files

Each Excel file:
- Contains multiple sheets
- Uses different layouts per payment system
- Reports role-wise transaction data

This function dynamically:
- Detects entity columns
- Handles role-specific column offsets
- Converts all sheets into a normalized row structure


In [31]:
def parse_npci_excel(file_path, month_name):
    """
    Parse a single RBI/NPCI entity-wise Excel file.

    Returns:
        DataFrame with columns:
        Month, Sheet, Entity, Role, Volume_Lakh, Value_Crore
    """
    xls = pd.ExcelFile(file_path)
    records = []

    for sheet_name in xls.sheet_names:

        # Identify supported payment systems
        sheet_type = next(
            (t for t in ['UPI', 'IMPS', 'NETC', 'NFS', 'AePS', 'CTS', 'BBPS']
             if t in sheet_name),
            None
        )
        if not sheet_type:
            continue

        df = pd.read_excel(xls, sheet_name=sheet_name, header=None)

        # Configuration per payment system
        configs = {
            'UPI':  {'roles': ['Remitter', 'Beneficiary'], 'vol_cols_offset': [0, 2], 'val_cols_offset': [1, 3]},
            'IMPS': {'roles': ['Remitter', 'Beneficiary'], 'vol_cols_offset': [0, 2], 'val_cols_offset': [1, 3]},
            'NETC': {'roles': ['Issuer', 'Acquirer'],      'vol_cols_offset': [0, 2], 'val_cols_offset': [1, 3]},
            'CTS':  {'roles': ['Presenting', 'Drawee'],    'vol_cols_offset': [0, 2], 'val_cols_offset': [1, 3]},
            'BBPS': {'roles': ['Customer OU', 'Biller OU'],'vol_cols_offset': [0, 2], 'val_cols_offset': [1, 3]},
            'AePS': {'roles': [
                        'Issuer Payment Transactions',
                        'Issuer Cash Withdrawal',
                        'Acquirer Payment Transactions',
                        'Acquirer Cash Withdrawal'
                     ],
                     'vol_cols_offset': [0, 2, 4, 6],
                     'val_cols_offset': [1, 3, 5, 7]},
            'NFS':  {'roles': [
                        'Issuer Payment Transactions',
                        'Issuer Cash Withdrawal',
                        'Acquirer Payment Transactions',
                        'Acquirer Cash Withdrawal'
                     ],
                     'vol_cols_offset': [0, 2, 4, 6],
                     'val_cols_offset': [1, 3, 5, 7]},
        }

        config = configs[sheet_type]

        entity_col = None
        data_start = None

        for i in range(20):
            row = df.iloc[i]
            row_str = [str(cell).strip().lower() for cell in row if pd.notna(cell)]

            if any('sr. no.' in s or 'entity name' in s or 'bank name' in s for s in row_str):
                for j in range(len(row)):
                    if 'sr. no.' in str(row[j]).lower():
                        entity_col = j + 1
                        break
                data_start = i + 1
                break

        # Fallback layouts
        if entity_col is None:
            if len(df.columns) > 0 and str(df.iloc[4, 0]).strip().isdigit():
                entity_col = 1
                data_start = 4
            elif len(df.columns) > 1 and str(df.iloc[4, 1]).strip().isdigit():
                entity_col = 2
                data_start = 4

        if entity_col is None:
            print(f"Could not detect layout for {sheet_name}")
            continue

        data_df = df.iloc[data_start:].reset_index(drop=True)

        for _, row in data_df.iterrows():
            if pd.isna(row[entity_col]):
                continue

            entity = str(row[entity_col]).strip()
            if not entity or 'total' in entity.lower():
                continue

            base_col = entity_col + 1

            for i in range(len(config['roles'])):
                vol_col = base_col + config['vol_cols_offset'][i]
                val_col = base_col + config['val_cols_offset'][i]

                vol = float(row[vol_col]) if len(row) > vol_col and pd.notna(row[vol_col]) else 0.0
                val = float(row[val_col]) if len(row) > val_col and pd.notna(row[val_col]) else 0.0

                records.append({
                    'Month': month_name,
                    'Sheet': sheet_name,
                    'Entity': entity,
                    'Role': config['roles'][i],
                    'Volume_Lakh': vol,
                    'Value_Crore': val
                })

    return pd.DataFrame(records)




## Combining All Monthly Files

This function:
- Iterates over a folder of monthly Excel files
- Parses each file
- Concatenates all records
- Writes a single normalized CSV


In [32]:
def combine_npci_excels(folder_path, output_csv='npci_complete_all_months.csv'):
    all_dfs = []

    for filename in sorted(os.listdir(folder_path)):
        if not filename.lower().endswith(('.xlsx', '.xls')):
            continue

        month = extract_month_year(filename)
        if not month:
            print(f"Skipping {filename}")
            continue

        print(f"Processing {filename} → {month}")
        df = parse_npci_excel(os.path.join(folder_path, filename), month)
        print(f"   {len(df)} rows extracted")

        if not df.empty:
            all_dfs.append(df)

    if not all_dfs:
        print("No data extracted.")
        return

    final = pd.concat(all_dfs, ignore_index=True)
    final.to_csv(output_csv, index=False)

    print(f"COMPLETE SUCCESS: {len(final)} rows written to '{output_csv}'")


In [33]:
combine_npci_excels(
    folder_path=r'../data/raw/entity_wise',
    output_csv='../data/processed/npci_entity_clean.csv'
)


Processing ENTITYWISEAUGUST2025A97939DA2773400EB042F300CD870831.XLSX → August 2025
   11914 rows extracted
Processing ENTITYWISEJULY2025DDC4C64BC69145CFB146A608B34BAFA2.XLSX → July 2025
   11920 rows extracted
Processing ENTITYWISEJUNE2025BA78CF4FD72F488D8FFF53C8EB64288F.XLSX → June 2025
   11872 rows extracted
Processing ENTITYWISESEPTEMBER2025DA18D89E70474C86B55A7BBC64E23654.XLSX → September 2025
   11972 rows extracted
Processing NPCIAPRIL2025C9B700AB0B004A4C8AA090259EB46442.XLSX → April 2025
   11792 rows extracted
Processing NPCIDATAAUGUST2024CA2B21275B1342E29B0BC56EC71AC3F3.XLSX → August 2024
   11612 rows extracted
Processing NPCIDATAJULY2424DDE1B8C48F44088038BBAF8E5EBC43.XLSX → July 2024
   11440 rows extracted
Processing NPCIDATAJUN24F685A5468ADE4F8AAAD721EC08B995A6.XLSX → June 2024
   11430 rows extracted
Processing NPCIDATASEPTEMBER2024F4347CC0969841F2BB416DA63EBE8D93.XLSX → September 2024
   11678 rows extracted
Processing NPCIDECEMBER2024CD667135FD57439196E98C55756267CB.XL

In [44]:
# Example: original read
file_path = "../data/processed/npci_entity_clean.csv"
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223250 entries, 0 to 223249
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Month        223250 non-null  object 
 1   Sheet        223250 non-null  object 
 2   Entity       223250 non-null  object 
 3   Role         223250 non-null  object 
 4   Volume_Lakh  223250 non-null  float64
 5   Value_Crore  223250 non-null  float64
dtypes: float64(2), object(4)
memory usage: 10.2+ MB


## Bank Name Normalization & Deduplication

Bank/entity names appear in multiple formats across the dataset, leading to duplicate entries during aggregation.

To address this:
- Entity names are cleaned using basic text standardization (case, spacing, suffix removal)
- Major banks are manually mapped to a single canonical name
- Fuzzy matching is used to merge near-duplicate names caused by spelling or ordering differences

This step ensures consistent bank-level aggregation and avoids inflated entity counts in downstream analysis.


In [35]:
# Unique entity count
df['Entity'].nunique()

5924

In [36]:
unique_banks = df['Entity'].unique()
print("All unique bank codes:")
for bank in unique_banks:
    print(bank)


All unique bank codes:
Yes Bank
State Bank of India
Axis Bank
HDFC Bank
Bank of Baroda
Union Bank Of India
Punjab National Bank
Canara Bank
ICICI Bank
Kotak Mahindra Bank
Airtel Payments Bank
Indian Bank
INDIA POST PAYMENTS BANK LIMITED
Bank Of India
Federal Bank
Indian Overseas Bank
Central Bank of India
Indusind Bank
UCO Bank
FINO Payment bank
Bank of Maharashtra
IDBI Bank
Tri O Tech Solutions Private Limited
Karnataka Bank
IDFC Bank
Unity Small Finance Bank Limited
The Jammu & Kashmir Bank Limited
Bandhan Bank limited
Karur Vysya Bank
RBL Bank
The South Indian Bank Limited
AU Small Finance Bank
JIO PAYMENTS BANK LTD
NSDL Payments Bank
Tamilnad Mercantile Bank
City Union Bank
Punjab and Sind Bank
UJJIVAN SMALL FINANCE BANK LIMITED
KERALA GRAMIN BANK
ONE MOBIKWIK SYSTEMS LIMITED
Baroda U.P. Bank
SURYODAY SMALL FINANCE BANK LIMITED
TELANGANA GRAMEENA BANK
ESAF SMALL FINANCE BANK LTD
Karnataka Gramin Bank
Maharashtra Gramin Bank
SBI Cards and Payment Services Limited
EQUITAS SMALL FINAN

In [41]:
import re
from thefuzz import fuzz, process


def clean_bank_name(name):
    """
    Standardizes bank/entity names by removing common noise,
    normalizing text, and handling major known banks.
    """
    if not isinstance(name, str) or pd.isna(name):
        return ""

    name = name.strip().upper()
    name = re.sub(r'\s+', ' ', name)

    # Remove common suffixes / prefixes seen in Indian bank names
    remove_patterns = [
        r'\bLTD\b.*$', r'\bLIMITED\b.*$', r'\bPRIVATE LIMITED\b.*$', r'\bPVT LTD\b.*$',
        r'\bCO[-\s]?OPERATIVE\b.*$', r'\bCO[-\s]?OP\b.*$', r'\bCOOPERATIVE\b.*$',
        r'\bBANK LIMITED\b.*$', r'\bBANK LTD\b.*$',
        r'\bSAHAKARI\b.*$', r'\bMARYADIT\b.*$', r'\bNIYAMIT\b.*$',
        r'\bSOUHARDA\b.*$', r'\bURBAN\b.*$', r'\bDISTRICT CENTRAL\b.*$',
        r'\bPEOPLES\b.*$', r'\bMERCHANTS\b.*$', r'\bGRAMIN\b.*$',
        r'\bIMPS\b.*$', r'-IMPS.*$', r'\bERSTWHILE\b.*$',
        r'^\bTHE\b\s*', r'^\bSHRI\b\s*', r'^\bSHREE\b\s*', r'^\bSRI\b\s*'
    ]

    for pattern in remove_patterns:
        name = re.sub(pattern, '', name, flags=re.IGNORECASE)

    # Keep only alphanumeric characters, spaces, and &
    name = re.sub(r'[^A-Z0-9\s&]', ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()

    # Manual standardization for major banks
    known_banks = {
        'HDFC': 'HDFC BANK',
        'ICICI': 'ICICI BANK',
        'AXIS': 'AXIS BANK',
        'SBI': 'STATE BANK OF INDIA',
        'STATE BANK': 'STATE BANK OF INDIA',
        'PUNJAB NATIONAL': 'PUNJAB NATIONAL BANK',
        'BANK OF BARODA': 'BANK OF BARODA',
        'UNION BANK': 'UNION BANK OF INDIA',
        'CANARA': 'CANARA BANK',
        'IDFC FIRST': 'IDFC FIRST BANK',
        'INDUSIND': 'INDUSIND BANK',
        'FEDERAL': 'FEDERAL BANK',
        'RBL': 'RBL BANK',
        'KOTAK': 'KOTAK MAHINDRA BANK',
        'PAYTM PAYMENTS': 'PAYTM PAYMENTS BANK',
        'AIRTEL PAYMENTS': 'AIRTEL PAYMENTS BANK',
        'INDIA POST PAYMENTS': 'INDIA POST PAYMENTS BANK'
    }

    for key, standard_name in known_banks.items():
        if key in name:
            return standard_name

    return name


# Apply basic cleaning
df['Entity'] = df['Entity'].apply(clean_bank_name)


def deduplicate_names(series, similarity_threshold=88):
    """
    Groups similar entity names using fuzzy matching
    and maps them to a single canonical name.
    """
    unique_values = series.unique().tolist()
    canonical_mapping = {}

    while unique_values:
        base = unique_values.pop(0)

        matches = process.extract(
            base,
            unique_values,
            scorer=fuzz.token_sort_ratio,
            limit=len(unique_values)
        )

        close_matches = [m[0] for m in matches if m[1] >= similarity_threshold]
        close_matches.append(base)

        canonical_name = min(close_matches, key=len)

        for val in close_matches:
            canonical_mapping[val] = canonical_name
            if val in unique_values:
                unique_values.remove(val)

    return canonical_mapping


# Create mapping and apply it
mapping = deduplicate_names(df['Entity'])
df['Entity'] = df['Entity'].map(mapping)

print("Unique banks after normalization:", df['Entity'].nunique())
print("\nTop 20 banks:")
print(df['Entity'].value_counts().head(20))


Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']


Unique banks after normalization: 1680

Top 20 banks:
Entity
                            4632
JILA                        4564
CENTRAL                     2534
DISTRICT                    2290
CENTRAL BANK OF INDIA       2178
JANATA                      1402
ZILA SAHAKRI BANK           1346
KARNATAKA BANK              1150
ZILA                         918
JAIN                         806
MAHESH                       748
NAGRIK                       728
VIJAY                        722
PALI CENTRAL                 688
UNION BANK OF INDIA          686
BANK OF INDIA                684
PUNE                         646
RAMA                         602
CITIZEN                      598
KOTA NAGRIK SAHKARI BANK     596
Name: count, dtype: int64


In [42]:
df.head()

,Month,Sheet,Entity,Role,Volume_Lakh,Value_Crore
0,August 2025,UPI,YES BANK,Remitter,1782.75231,25900.453123
1,August 2025,UPI,YES BANK,Beneficiary,80877.76646,247763.862678
2,August 2025,UPI,STATE BANK OF INDIA,Remitter,50282.91989,645452.945541
3,August 2025,UPI,STATE BANK OF INDIA,Beneficiary,20507.78499,471722.817698
4,August 2025,UPI,AXIS BANK,Remitter,8843.33913,134418.481534


In [ ]:
# Unique entity count
df['Entity'].nunique()

In [43]:
unique_banks = df['Entity'].unique()
print("All unique bank codes:")
for bank in unique_banks:
    print(bank)


All unique bank codes:
YES BANK
STATE BANK OF INDIA
AXIS BANK
IDFC BANK
BANK OF BARODA
UNION BANK OF INDIA
PUNJAB NATIONAL BANK
CANARA BANK
ICICI BANK
KOTAK MAHINDRA BANK
AIRTEL PAYMENT BANK
INDIAN BANK
INDIA POST PAYMENT BANK
BANK OF INDIA
FEDERAL BANK
INDIAN OVERSEAS BANK
INDUSIND BANK
UCO BANK
JIO PAYMENTS BANK
BANK OF MAHARASHTRA
IDBI BANK
TRI O TECH SOLUTIONS PRIVATE
KARNATAKA BANK
UNITY SMALL FINANCE BANK
JAMMU AND KASHMIR
BANDHAN BANK
KARUR VYSYA BANK
RBL BANK
SOUTH INDIAN BANK
AU SMALL FINANCE BANK
NSDL PAYMENTS BANK
TAMILNAD MERCANTILE BANK
PUNJAB & SIND BANK
UJJIVAN SMALL FINANCE BANK
KERALA
ONE MOBIKWIK SYSTEMS
BARODA U P BANK
SURYODAY SMALL FINANCE BANK
TELANGANA GRAMEENA BANK
ESAF SMALL FINANCE BANK
MAHARASHTRA
EQUITAS SMALL FINANCE BANK
ANDHRA PRADESH GRAMEENA BANK
RAJASTHAN MARUDHARA
DBS BANK INDIA
RAMA
SLICE SMALL FINANCE BANK
SARVA HARYANA
PRATHMA UP
STANDARD CHARTERD BANK
CITIBANK CORPORATE
DAKSHIN BIHAR
HSBC BANK
CHATTISGARH
BARODA RAJASTHAN KSHETRIYA
UTKARSH SMALL F

In [45]:
df.to_csv(file_path, index=False)

### Output Dataset

Final schema:
- Month
- Sheet
- Entity
- Role
- Volume_Lakh
- Value_Crore

This dataset is:
- Fully normalized
- Entity-granular
- Suitable for reconciliation with PSDDP
- Ready for market share and anomaly analysis